In [ ]:
class Normalize:

    @staticmethod
    def normalize(Y, R):
        m, n = Y.shape
        YMean = np.zeros((m, 1))
        YNorm = np.zeros(Y.shape)

        for i in range(0, m):
            idx = np.where(R[i, :] == 1)
            YMean[i] = np.mean(Y[i, idx])
            YNorm[i, idx] = Y[i, idx] - YMean[i]

        return YNorm, YMean

In [ ]:
class CollaborativeFilter:

    @staticmethod
    def cost(params, *args):
        # Args
        Y, R, numUser, numMovies, numFeature, lam, batchSize, anim, costs, counts = args
        X = np.reshape(params[0:numMovies * numFeature], (numMovies, numFeature))
        Theta = np.reshape(params[numMovies * numFeature:], (numUser, numFeature))

        J = sum(sum(R * np.square((Theta.dot(X.T)).T - Y))) / 2 + (lam * sum(sum(np.square(Theta)))) / 2 + (
                lam * sum(sum(np.square(X)))) / 2
        
        costs.append(np.divide(J, 1000))
        counts.append(len(costs))
        
        #anim.plotScores(np.array(counts), np.array(costs))
        
        return J

    @staticmethod
    def gradient(params, *args):
        # Args
        Y, R, numUser, numMovies, numFeature, lam, batchSize, anim, costs, counts = args
        X = np.reshape(params[0:numMovies * numFeature], (numMovies, numFeature))
        Theta = np.reshape(params[numMovies * numFeature:], (numUser, numFeature))

        grad = np.multiply((np.dot(X, Theta.T) - Y), R)
        # gradients
        X_grad = np.dot(grad, Theta) + lam * X
        Theta_grad = np.dot(grad.T, X) + lam * Theta

        params = np.concatenate((X_grad, Theta_grad)).ravel()

        return params
    
    @staticmethod
    def stochasticGradient(params, *args):
        
        Y, R, numUser, numMovies, numFeature, lam, batchSize, anim, costs, counts = args
        X = np.reshape(params[0:numMovies * numFeature], (numMovies, numFeature))
        Theta = np.reshape(params[numMovies * numFeature:], (numUser, numFeature))

        X_grad = np.zeros(X.shape)
        Theta_grad = np.zeros(Theta.shape)
        
        for i in range(0, numMovies, batchSize):
            idx = np.where(R[i:i+batchSize, :] == 1)
            ThetaTemp = Theta[idx[1]]
            YTemp = Y[i:i+batchSize, idx[1]]
            
            X_grad[i:i+batchSize, :] = (np.dot(X[i:i+batchSize, :], ThetaTemp.T) - YTemp).dot(ThetaTemp) + lam * X[i:i+batchSize, :]

        for i in range(0, numUser, batchSize):
            idx = np.where(R[:, i:i+batchSize] == 1)
            XTemp = X[idx[0]]
            YTemp = Y[idx[0], i:i+batchSize]

            Theta_grad[i:i+batchSize, :] = (np.dot(XTemp,Theta[i:i+batchSize, :].T) - YTemp).T.dot(XTemp) + lam * Theta[i:i+batchSize, :]

        params = np.concatenate((X_grad, Theta_grad)).ravel()

        return params